In [20]:
import pandas as pd
import random
import numpy as np
import copy
import networkx as nx
import matplotlib.pyplot as plt
import itertools
import collections
from collections import deque 

In [78]:
class QuantumNetworks():
    def __init__(self, pGen, pSwap, cutOffAge, maxQubits, initialEdges):
        self.pGen = pGen
        self.pSwap = pSwap
        self.cutOffAge = cutOffAge
        self.maxQubits = maxQubits
        self.initialEdges = initialEdges
        self.currentEdges = {}

    def __generateEntanglement(self, node1, node2):
        edge = tuple(sorted([node1, node2]))
        # Count current entanglements for each node
        node1_count = sum(len(links) for e, links in self.currentEdges.items() if node1 in e)
        node2_count = sum(len(links) for e, links in self.currentEdges.items() if node2 in e)
        
        if node1_count < self.maxQubits and node2_count < self.maxQubits:
            if edge not in self.currentEdges:
                self.currentEdges[edge] = deque([0])
            else:
                self.currentEdges[edge].appendleft(0)
        else:
            print(f"Node {node1} or {node2} has reached the maximum number of entanglements.")

    def randomlyGenerateEntanglementsGlobally(self):
        for edge in self.initialEdges:
            if random.random() < self.pGen:
                self.__generateEntanglement(*edge)
                
    def __discardEntanglement(self, edge):
        if edge in self.currentEdges and len(self.currentEdges[edge]) > 0:
            self.currentEdges[edge].pop()  # Remove oldest entanglement
            
        if not self.currentEdges[edge]:  # If no more entanglements
            del self.currentEdges[edge]

    def performEntanglementSwaping(self, edge1, edge2):
        # Check if edges share a common node and have available entanglements
        if (edge1 in self.currentEdges and edge2 in self.currentEdges and 
            len(self.currentEdges[edge1]) > 0 and len(self.currentEdges[edge2]) > 0):
            
            # Find the common node and outer nodes
            common_nodes = set(edge1) & set(edge2)
            if len(common_nodes) != 1:
                return  # Edges must share exactly one common node
            
            all_nodes = list(set(edge1) | set(edge2))
            outer_nodes = [node for node in all_nodes if node not in common_nodes]
            new_edge = tuple(sorted(outer_nodes))
            
            # Attempt swapping with probability pSwap
            if random.random() < self.pSwap:
                # Store ages before discarding
                age1 = self.currentEdges[edge1][0]
                age2 = self.currentEdges[edge2][0]
                
                # Discard entanglements from input edges
                self.__discardEntanglement(edge1)
                self.__discardEntanglement(edge2)
                
                # Create or add entanglement to the new edge with max age
                new_age = max(age1, age2)
                if new_edge not in self.currentEdges:
                    self.currentEdges[new_edge] = deque([new_age])
                else:
                    # Add new age while maintaining ascending order
                    ages = list(self.currentEdges[new_edge])
                    ages.append(new_age)
                    ages.sort()
                    self.currentEdges[new_edge] = deque(ages)
        else:
            print(f"No entanglement found for edges {edge1} and {edge2}")

In [79]:
initialEdges = [(0, 1), (1, 2), (2, 3)]
network = QuantumNetworks(pGen=0.5, pSwap=1, cutOffAge=1, maxQubits=3, initialEdges=initialEdges)
network.randomlyGenerateEntanglementsGlobally()
network.randomlyGenerateEntanglementsGlobally()
print(network.currentEdges)
network.performEntanglementSwaping((2,3), (1, 2))
print(network.currentEdges)

{(0, 1): deque([0]), (1, 2): deque([0]), (2, 3): deque([0])}
{(0, 1): deque([0]), (1, 3): deque([0])}
